In [3]:
"""
本算法将通过 均值逼近算法 解决第二类公平性指派问题
65
[(1, 2), (3, 6), (5, 4), (6, 3), (0, 0), (2, 1), (4, 5)]
"""

# 初始参数设置
# 工作负荷矩阵
# WorkloadMatrtix = [
#     [9,11,3,6,6,7,4],
#     [10,9,11,5,6,3,6],
#     [8,10,5,6,4,7,1],
#     [6,8,10,4,9,5,8],
#     [11,1,9,8,7,11,8],
#     [4,1,3,2,7,0,3],
#     [4,6,2,9,8,5,4],
# ]

WorkloadMatrtix = 5 + np.random.normal(loc=10, scale=5, size=[100, 100])

# 代理数目
# N = 7
N = 100
# 步长补偿
step_compensation = 1
# 收敛判断常数
gamma = 1

def demo():
    """
    测试用例
    :return:
    """
    profit_matrix = [
        [62, 75, 80, 93, 0, 97],
        [75, 0, 82, 85, 71, 97],
        [80, 75, 81, 0, 90, 97],
        [78, 82, 0, 80, 50, 98],
        [0, 85, 85, 80, 85, 99],
        [65, 75, 80, 75, 68, 0]]
    hungarian = Hungarian(profit_matrix, is_profit_matrix=True)
    hungarian.calculate()
    print("Expected value:\t\t523")
    print("Calculated value:\t", hungarian.get_total_potential())  # = 523
    print("Expected results:\n\t[(0, 3), (2, 4), (3, 0), (5, 2), (1, 5), (4, 1)]")
    print("Results:\n\t", hungarian.get_results())
    print("-" * 80)

def calculateCav(wlMatrtix, li, n=1):
    """
    本函数用于计算某个可行解下的平均工作负荷
    :param wlMatrtix: ndarray, 工作矩阵矩阵
    :param li: 某个可行解
    :return:
    """
    # print('正在计算新的平均工作负荷：')
    # print('当前解',li)
    # print(wlMatrtix)
    # 求平均值
    if n == 1:
        n = wlMatrtix.shape[0]
    value = 0
    for row, column in li:
        # print(row, column, wlMatrtix[row, column])
        value += wlMatrtix[row, column]
    ave = value / n
    # print("ave = ", ave)
    # 求公平程度
    value = 0
    for row, column in li:
        value += (ave - wlMatrtix[row, column]) ** 2
    return value, ave

def FAP():
    """
    均值逼近算法的核心代码
    """
    # STEP1：确定工作负荷均值
    wlMatrtix = np.array(WorkloadMatrtix, dtype=np.int32)
    c_min = wlMatrtix.min()
    c_max = wlMatrtix.max()
    c      = c_min
    c_fore = c_min - 1
    n      = N

    # STEP2:确定迭代步长
    li = []
    for i in range(n):
        for j in range(n):
            li.append(wlMatrtix[i][j])
    li = sorted(li)
    subnum = 9999999
    for i in range(n - 1):
        if (li[i + 1] - li[i]) < subnum:
            subnum = li[i + 1] - li[i]
    delta = max((subnum + 1) / n,step_compensation)   # 这里要+1，避免出现0
    # print('delta = ', delta)
    # i = 0
    # 保存结果
    result_c = []                   # 保存每次收敛的均值
    result_min = 999999             # 保存当前的最小均值
    result_x = []                   # 保存最小均值对应的可行解
    result_ave = 0                  # 保存最终的工作负荷均值
    # result_
    i = 0
    while c < c_max and i < N * 10:
        print("start loop: ", i)
        c_initial = c               # 保存当前c
        # STEP3: 调用匈牙利算法
        j = 0
        while True:
            print("looping ... ... ")
            workMatrtix = np.array(wlMatrtix,dtype=np.float32)
            # 根据算法（式子1.13）规则，要在这里调整工作负荷矩阵。匈牙利算法的接受值是一个标准工作负荷矩阵
            workMatrtix = (workMatrtix - c) ** 2
            hungarian = Hungarian(workMatrtix, is_profit_matrix=False, is_FAP=True, FAP_num=c)
            hungarian.calculate()
            # f_r    = hungarian.get_total_potential()    # 目标函数值
            x      = hungarian.get_results()            # 可行解
            # print('第',i,'次循环：可行解', x)
            # i+=1
            c_hat, c_ave  = calculateCav(wlMatrtix, x, n)      # 计算出了新的c
            result_c.append(c_hat)
            # print(result_c)
            if c_hat < result_min or c==c_min:
                result_min = c_hat                      # 更新最小均值
                result_x = x                            # 更新最小可行解
                result_ave = c_ave

            # print('当前均值：', c_hat)
            if abs(c - c_hat) > gamma:
                # 未收敛：更新c
                # print("not 收敛")
                c = c_hat
            else:
                # 收敛了就要找机会break
                if c_fore == c_hat:     # 原论文在这里的描述比较模糊，我认为应该是这样的代码逻辑吧
                    c = c_initial + delta
                    break
                else:
                    c_fore = c_hat
                    c = c_hat + delta
                    break
            if j==10:
                break
            j += 1
            # end while
        i += 1
        # end while
        # STEP4：更换新的工作负荷均值
        # c = c_initial + delta
    print("result:")
    print("最佳目标函数: ",result_min)
    print("此时对应解: ",result_x)
    print("此时对应的平均工作负荷：",result_ave)
    print(result_c)
    # end while

In [2]:
'''
匈牙利算法

'''
import numpy as np
import collections
import time

class Hungarian():
    """
    """

    def __init__(self, input_matrix=None, is_profit_matrix=False, is_FAP=False, FAP_num = 0):
        """
        input_matrix     输入为一个二维嵌套列表, 也可以直接输入ndarray类型的数据
        is_profit_matrix 代表输入是否为收益矩阵。利益矩阵（需要使利益最大化），消费矩阵（需要使消费最小化）
        is_FAP           代表是否启动公平性分配模式，此时会影响计算结果
        FAP_num          FAP模式下的一个参数
        以下为这个类中自带的一些参数：
        self._input_matrix  输入矩阵
        self._maxColumn     列数
        self._maxRow        行数
        self._cost_matrix   最终矩阵
        self._size
        self._shape

        """
        if input_matrix is not None:
            if isinstance(input_matrix, np.ndarray):
                my_matrix = input_matrix
            else:
                # 保存输入
                my_matrix = np.array(input_matrix)
            self._input_matrix = np.array(input_matrix)
            self._maxColumn = my_matrix.shape[1]
            self._maxRow = my_matrix.shape[0]
            self._is_FAP = is_FAP
            self._Fap_num = FAP_num

            # 本算法必须作用于方阵，如果不为方阵则填充0变为方阵
            matrix_size = max(self._maxColumn, self._maxRow)
            pad_columns = matrix_size - self._maxRow
            pad_rows = matrix_size - self._maxColumn
            my_matrix = np.pad(my_matrix, ((0, pad_columns), (0, pad_rows)), 'constant', constant_values=(0))

            # 如果需要，则转化为消费矩阵
            if is_profit_matrix:
                my_matrix = self.make_cost_matrix(my_matrix)

            self._cost_matrix = my_matrix
            self._size = len(my_matrix)
            self._shape = my_matrix.shape

            # 存放算法结果
            self._results = []
            self._totalPotential = 0
        else:
            self._cost_matrix = None

    def make_cost_matrix(self, profit_matrix):
        """
        利益矩阵转化为消费矩阵，输出为numpy矩阵'''
        消费矩阵 = 利益矩阵最大值组成的矩阵 - 利益矩阵
        """
        matrix_shape = profit_matrix.shape
        offset_matrix = np.ones(matrix_shape, dtype=int) * profit_matrix.max()
        cost_matrix = offset_matrix - profit_matrix
        return cost_matrix

    def get_results(self):
        """获取算法结果"""
        return self._results

    def calculate(self):
        """
        实施匈牙利算法的函数
        """
        result_matrix = self._cost_matrix.copy()

        # 步骤 1: 矩阵每一行减去本行的最小值 [初等行变换]
        for index, row in enumerate(result_matrix):
            result_matrix[index] -= row.min()

        # 步骤 2: 矩阵每一列减去本行的最小值 [初等列变换]
        for index, column in enumerate(result_matrix.T):
            result_matrix[:, index] -= column.min()
        # print('步骤2结果 ',result_matrix)
        # 步骤 3： 使用最少数量的划线覆盖矩阵中所有的0元素
        # 如果划线总数不等于矩阵的维度需要进行矩阵调整并重复循环此步骤
        total_covered = 0
        while total_covered < self._size:
            # time.sleep(1)
            # 使用最少数量的划线覆盖矩阵中所有的0元素同时记录划线数量
            cover_zeros = CoverZeros(result_matrix)
            single_zero_pos_list = cover_zeros.calculate()
            covered_rows = cover_zeros.get_covered_rows()
            covered_columns = cover_zeros.get_covered_columns()
            total_covered = len(covered_rows) + len(covered_columns)

            # 如果划线总数不等于矩阵的维度需要进行矩阵调整（需要使用未覆盖处的最小元素）
            if total_covered < self._size:
                result_matrix = self._adjust_matrix_by_min_uncovered_num(result_matrix, covered_rows, covered_columns)
        # 元组形式结果对存放到列表
        self._results = single_zero_pos_list
        # 计算总期望结果
        # ---------------------------------常规-------------------------------
        # 常规匈牙利算法情况下的目标函数
        value = 0
        if not self._is_FAP:
            for row, column in single_zero_pos_list:
                value += self._input_matrix[row, column]
        else:
            for row, column in single_zero_pos_list:
                value += (self._input_matrix[row, column] - self._Fap_num) ** 2
        self._totalPotential = value

    def get_total_potential(self):
        return self._totalPotential

    def _adjust_matrix_by_min_uncovered_num(self, result_matrix, covered_rows, covered_columns):
        """计算未被覆盖元素中的最小值（m）,未被覆盖元素减去最小值m,行列划线交叉处加上最小值m"""
        adjusted_matrix = result_matrix
        # 计算未被覆盖元素中的最小值（m）
        elements = []
        for row_index, row in enumerate(result_matrix):
            if row_index not in covered_rows:
                for index, element in enumerate(row):
                    if index not in covered_columns:
                        elements.append(element)
        min_uncovered_num = min(elements)
        # print('min_uncovered_num:',min_uncovered_num)
        # 未被覆盖元素减去最小值m
        for row_index, row in enumerate(result_matrix):
            if row_index not in covered_rows:
                for index, element in enumerate(row):
                    if index not in covered_columns:
                        adjusted_matrix[row_index, index] -= min_uncovered_num
        # print('未被覆盖元素减去最小值m',adjusted_matrix)

        # 行列划线交叉处加上最小值m
        for row_ in covered_rows:
            for col_ in covered_columns:
                # print((row_,col_))
                adjusted_matrix[row_, col_] += min_uncovered_num
        # print('行列划线交叉处加上最小值m',adjusted_matrix)

        return adjusted_matrix


class CoverZeros():
    """
    使用最少数量的划线覆盖矩阵中的所有零
    输入为numpy方阵
    """

    def __init__(self, matrix):
        # 找到矩阵中零的位置（输出为同维度二值矩阵，0位置为true，非0位置为false）
        self._zero_locations = (matrix == 0)
        self._zero_locations_copy = self._zero_locations.copy()
        self._shape = matrix.shape

        # 存储划线盖住的行和列
        self._covered_rows = []
        self._covered_columns = []

    def get_covered_rows(self):
        """返回覆盖行索引列表"""
        return self._covered_rows

    def get_covered_columns(self):
        """返回覆盖列索引列表"""
        return self._covered_columns

    def row_scan(self, marked_zeros):
        '''扫描矩阵每一行，找到含0元素最少的行，对任意0元素标记（独立零元素），划去标记0元素（独立零元素）所在行和列存在的0元素'''
        min_row_zero_nums = [9999999, -1]
        for index, row in enumerate(self._zero_locations_copy):  # index为行号
            row_zero_nums = collections.Counter(row)[True]
            if row_zero_nums < min_row_zero_nums[0] and row_zero_nums != 0:
                # 找最少0元素的行
                min_row_zero_nums = [row_zero_nums, index]
        # 最少0元素的行
        row_min = self._zero_locations_copy[min_row_zero_nums[1], :]
        # 找到此行中任意一个0元素的索引位置即可
        row_indices, = np.where(row_min)
        # 标记该0元素
        # print('row_min',row_min)
        marked_zeros.append((min_row_zero_nums[1], row_indices[0]))
        # 划去该0元素所在行和列存在的0元素
        # 因为被覆盖，所以把二值矩阵_zero_locations中相应的行列全部置为false
        self._zero_locations_copy[:, row_indices[0]] = np.array([False for _ in range(self._shape[0])])
        self._zero_locations_copy[min_row_zero_nums[1], :] = np.array([False for _ in range(self._shape[0])])

    def calculate(self):
        '''进行计算'''
        # 储存勾选的行和列
        ticked_row = []
        ticked_col = []
        marked_zeros = []
        # 1、试指派并标记独立零元素
        while True:
            # print('_zero_locations_copy',self._zero_locations_copy)
            # 循环直到所有零元素被处理（_zero_locations中没有true）
            if True not in self._zero_locations_copy:
                break
            self.row_scan(marked_zeros)

        # 2、无被标记0（独立零元素）的行打勾
        independent_zero_row_list = [pos[0] for pos in marked_zeros]
        ticked_row = list(set(range(self._shape[0])) - set(independent_zero_row_list))
        # 重复3,4直到不能再打勾
        TICK_FLAG = True
        while TICK_FLAG:
            # print('ticked_row:',ticked_row,'   ticked_col:',ticked_col)
            TICK_FLAG = False
            # 3、对打勾的行中所含0元素的列打勾
            for row in ticked_row:
                # 找到此行
                row_array = self._zero_locations[row, :]
                # 找到此行中0元素的索引位置
                for i in range(len(row_array)):
                    if row_array[i] == True and i not in ticked_col:
                        ticked_col.append(i)
                        TICK_FLAG = True

            # 4、对打勾的列中所含独立0元素的行打勾
            for row, col in marked_zeros:
                if col in ticked_col and row not in ticked_row:
                    ticked_row.append(row)
                    FLAG = True
        # 对打勾的列和没有打勾的行画画线
        self._covered_rows = list(set(range(self._shape[0])) - set(ticked_row))
        self._covered_columns = ticked_col

        return marked_zeros

In [4]:
if __name__ == '__main__':
    FAP()

start loop:  0
result:
最佳目标函数:  446.7600000000003
此时对应解:  [(0, 99), (4, 60), (8, 3), (9, 69), (10, 43), (11, 85), (15, 75), (2, 40), (16, 16), (18, 57), (19, 70), (20, 4), (21, 61), (23, 98), (24, 7), (25, 21), (26, 28), (27, 30), (13, 87), (30, 55), (33, 23), (34, 64), (32, 8), (36, 45), (39, 58), (40, 81), (17, 25), (41, 2), (42, 22), (43, 89), (45, 9), (47, 6), (52, 17), (53, 62), (44, 42), (63, 77), (66, 31), (64, 54), (67, 34), (68, 83), (69, 79), (28, 46), (70, 26), (72, 13), (29, 10), (73, 38), (37, 14), (59, 80), (74, 47), (77, 84), (3, 53), (50, 92), (79, 20), (57, 91), (58, 65), (31, 95), (83, 29), (1, 68), (84, 39), (85, 48), (87, 36), (54, 0), (88, 66), (89, 52), (46, 76), (6, 63), (82, 88), (75, 94), (62, 56), (80, 12), (60, 93), (38, 59), (91, 67), (56, 1), (55, 33), (90, 19), (61, 15), (92, 24), (94, 44), (95, 50), (93, 72), (96, 51), (97, 90), (51, 74), (22, 71), (81, 5), (78, 49), (86, 97), (48, 32), (5, 27), (49, 18), (99, 86), (7, 37), (98, 11), (71, 78), (14, 82), (